<a href="https://colab.research.google.com/github/applekoong/DeepStudy/blob/master/adult.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb
import xgboost as xgb
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [0]:
columns = ['age',  # continuous.
'workclass',  # Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
'fnlwgt',  # continuous.
'education',  # Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
'education-num',  # continuous.
'marital-status',  # Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
'occupation',  # Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
'relationship',  # Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
'race',  # White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
'sex',  # Female, Male.
'capital-gain',  # continuous.
'capital-loss',  # continuous.
'hours-per-week',  # continuous.
'native-country',  # United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
'Y']

In [0]:
def strip(text):
    try:
        return text.strip()
    except AttributeError:
        return text

def make_int(text):
    return int(text.strip('" '))

In [0]:
train = pd.read_csv('./adult.data', names=columns, sep=' *, *')
test = pd.read_csv('./adult.test', names=columns, sep=' *, *')
train.head()
test.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Y
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


In [0]:
train.shape, test.shape

((32561, 15), (16281, 15))

In [0]:
train2 = pd.get_dummies(train)
test2 = pd.get_dummies(test)
train2.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,Y_<=50K,Y_>50K
0,39,77516,13,2174,0,40,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,50,83311,13,0,0,13,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,38,215646,9,0,0,40,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
3,53,234721,7,0,0,40,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
4,28,338409,13,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [0]:
test2.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,Y_<=50K.,Y_>50K.
0,25,226802,7,0,0,40,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,38,89814,9,0,0,50,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,28,336951,12,0,0,40,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
3,44,160323,10,7688,0,40,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,18,103497,10,0,0,30,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0


In [0]:
print(len(train2.columns))
print(len(test2.columns))

test2.insert(81, 'native-country_Holand-Netherlands', 0)

print(len(train2.columns))
print(len(test2.columns))

110
109
110
110


In [0]:
for i, col in enumerate(test2.columns):
    if train2.columns[i] != col:
        print(i, train2.columns[i], '**', col)

108 Y_<=50K ** Y_<=50K.
109 Y_>50K ** Y_>50K.


In [0]:
test2.rename(columns={'Y_<=50K.':'Y_<=50K',
                          'Y_>50K.':'Y_>50K'}, 
                 inplace=True)

In [0]:
for i, col in enumerate(test2.columns):
    if train2.columns[i] != col:
        print(i, train2.columns[i], '**', col)

In [0]:
test2.columns

Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week', 'workclass_?', 'workclass_Federal-gov',
       'workclass_Local-gov', 'workclass_Never-worked',
       ...
       'native-country_Scotland', 'native-country_South',
       'native-country_Taiwan', 'native-country_Thailand',
       'native-country_Trinadad&Tobago', 'native-country_United-States',
       'native-country_Vietnam', 'native-country_Yugoslavia', 'Y_<=50K',
       'Y_>50K'],
      dtype='object', length=110)

In [0]:
Mmscaler = MinMaxScaler()
train3 = Mmscaler.fit_transform(train2)
test3 = Mmscaler.transform(test2)

x_train = train3[:, :-2]
y_train = train3[:, -2:]
x_test = test3[:, :-2]
y_test = test3[:, -2:]
print(x_test.shape, y_test.shape)

(16281, 108) (16281, 2)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [0]:
batch_size = 128
num_classes = 2
epochs = 13

row, col = x_train.shape

In [0]:
input_shape = (col,)

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=input_shape))
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 128)               13952     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 258       
Total params: 30,722
Trainable params: 30,722
Non-trainable params: 0
_________________________________________________________________
Train on 32561 samples, validate on 16281 samples
Epoch 1/13
32561/32561 [==============================] - 1s 44us/step - loss: 0.3911 - 

In [0]:
y_pred = model.predict(x_test)
y_pred1 = y_pred[:, 0]

In [0]:
lgbm = lgb.LGBMClassifier()
rf = RandomForestClassifier()

In [0]:
lgb_train = lgb.Dataset(x_train, y_train[:, 0])
lgb_eval = lgb.Dataset(x_test, y_test[:, 0], reference=lgb_train)

In [0]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
#                 valid_sets=lgb_eval,
#                 early_stopping_rounds=5
               )

In [0]:
y_pred2 = gbm.predict(x_test, num_iteration=gbm.best_iteration, )
y_pred = np.where(y_pred2 > 0.5, 1, 0)
print(accuracy_score(y_test[:, 0], y_pred))

0.8745777286407469


In [0]:
rfm = rf.fit(x_train, y_train[:, 0])

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [0]:
y_pred3 = rfm.predict(x_test)
print(accuracy_score(y_test[:, 0], y_pred3))

0.8401203857256925


In [0]:
def stack_layer(x_train):
    x_train1 = model.predict(x_train)
    x_train1 = x_train1[:, 0]
    x_train2 = gbm.predict(x_train)
    x_train3 = rf.predict_proba(x_train)
    x_train3 = x_train3[:, 0]
#     print(x_train1.shape, x_train2.shape, x_train3.shape)
    x_stack = np.stack([x_train1, x_train2, x_train3], axis=1)
#     x_stack.shape
    return x_stack

In [0]:
x_stack = stack_layer(x_train)
x_stack.shape

(32561, 3)

In [0]:
xgbm = xgb.XGBClassifier(n_estimators=400)
xgbm.fit(x_stack, y_train[:, 0])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=400,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [0]:
x_test_stack = stack_layer(x_test)

In [0]:
y_pred_ensemble = xgbm.predict(x_test_stack)
y_pred_ensemble.shape
print(accuracy_score(y_pred_ensemble, y_test[:, 0]))

0.8344696271727781
